# Studying glass classification

This is the list of all the fields.

* Id number: 1 to 214 (removed from CSV file)
* RI: refractive index
* Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
* Mg: Magnesium
* Al: Aluminum
* Si: Silicon
* K: Potassium
* Ca: Calcium
* Ba: Barium
* Fe: Iron
* Type of glass: (class attribute)
  1 building_windows_float_processed
  2 building_windows_non_float_processed
  3 vehicle_windows_float_processed
  4 vehicle_windows_non_float_processed (none in this database)
  5 containers
  6 tableware
  7 headlamps


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Loading dataset and showing some records

In [ ]:
df = pd.read_csv('../input/glass.csv')

df.sample(5)

In [ ]:
X = df.drop(['Type'], axis=1)
Y = df['Type']


How the features influence the classification

In [ ]:
df.corr()['Type'].abs().sort_values(ascending=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import math

X_tr, X_ts, y_tr, y_ts = train_test_split(X, Y, test_size=0.10, random_state=42)

rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
param_grid = { "criterion" : ["gini", "entropy"]
              , "min_samples_leaf" : [1, 5, 10]
              , "min_samples_split" : [2, 4, 10, 12, 16]
              , "n_estimators": [100, 125, 200]}
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
gs = gs.fit(X_tr, y_tr)


In [ ]:
print(gs.best_score_)
print(gs.best_params_)


In [ ]:
rf = RandomForestClassifier( criterion='entropy', 
                             n_estimators=125,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)

rf.fit(X_tr, y_tr)
pred = rf.predict(X_ts)

score = rf.score(X_ts, y_ts)
err = math.sqrt(((pred - y_ts)**2).mean())
print("Error: %.3f Score: %.3f" % (err, score))


In [ ]:
#SKJ
rf = RandomForestClassifier( criterion='gini', 
                             n_estimators=200,
                             min_samples_split=2,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)

rf.fit(X_tr, y_tr)
pred = rf.predict(X_ts)

score = rf.score(X_ts, y_ts)
err = math.sqrt(((pred - y_ts)**2).mean())
print("Error: %.3f Score: %.3f" % (err, score))


This is the features importances for the algorithm

In [ ]:
pd.concat((pd.DataFrame(X.columns, columns = ['variable']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]